# Feature Engineering

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import text

from utils import setup_import

setup_import()

from fraud_detection.data.database import get_db_engine

plt.style.use("ggplot")
sns.set_palette("Set2")

engine = get_db_engine()

%load_ext autoreload
%autoreload 2

ROOT: /workspace/fraud-ml-pipeline
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. User Action Features (RFM Analysis)
- R (Recency - 直近性): 最後の取引からどれくらい時間が経ったか？<br>
    - (不正の観点): 短時間に連続した取引は、カードが乗っ取られている兆候かもしれない。
- F (Frequency - 頻度): ある一定期間内にどれくらいの頻度で取引しているか？<br>
    - (不正の観点): 普段より明らかに取引頻度が高い場合、不正の可能性がある。
- M (Monetary - 金額): 取引金額が平常時と比べてどう変化したか？<br>
    - (不正の観点): 普段は買わないような高額商品の購入は、不正の兆候かもしれない。



### Recency Features

In [22]:
query_r = text("""
    SELECT
        cc_num,
        trans_date,
        trans_time,
        amt,
        
        EXTRACT(EPOCH FROM(
            (trans_date || ' ' || trans_time)::timestamp - 
            LAG((trans_date || ' ' || trans_time)::timestamp, 1) OVER (
                PARTITION BY cc_num
                ORDER BY trans_date, trans_time
            )
        )) AS time_since_last_transaction_sec
    FROM
        transactions
    LIMIT 100;
""")

df_recency = pd.read_sql_query(sql=query_r, con=engine)
df_recency.head(20)

,cc_num,trans_date,trans_time,amt,time_since_last_transaction_sec
0,60400268763,2024-01-01,02:34:47,59.39,NaN
1,60400268763,2024-01-01,09:56:16,8.33,26489.0
2,60400268763,2024-01-04,05:09:17,7.51,241981.0
3,60400268763,2024-01-04,08:59:03,12.42,13786.0
4,60400268763,2024-01-06,09:04:15,64.03,173112.0
5,60400268763,2024-01-07,01:18:13,2.98,58438.0
6,60400268763,2024-01-07,10:18:50,106.24,32437.0
7,60400268763,2024-01-09,02:31:45,43.13,144775.0
8,60400268763,2024-01-09,08:25:07,67.34,21202.0
9,60400268763,2024-01-11,04:00:29,5.63,156922.0


### Frequecy and Monetary
- ある取引から遡り、過去24時間、1週間などに何回取引があったか
- その平均金額

「**Verocity features**」とも呼ばれる。

In [23]:
query_f = text("""
    SELECT
        cc_num,
        (trans_date || ' ' || trans_time)::timestamp AS trans_ts,
        amt,
        
        -- Feature 1: Recency
        EXTRACT(EPOCH FROM (
                (trans_date || ' ' || trans_time)::timestamp -
                LAG((trans_date || ' ' || trans_time)::timestamp, 1) OVER w
            )) AS time_since_last_transaction_sec,
        
        -- Feature 2: Frequency and Monetary
        COUNT(*) OVER (w RANGE BETWEEN INTERVAL '24 hours' PRECEDING AND CURRENT ROW) AS transactions_in_last_24h,
        COUNT(*) OVER (w RANGE BETWEEN INTERVAL '6 hours' PRECEDING AND CURRENT ROW) AS transactions_in_last_6h,
        AVG(amt) OVER (w RANGE BETWEEN INTERVAL '24 hour' PRECEDING AND CURRENT ROW) AS avg_amt_in_last_24h,
        COUNT(*) OVER (w RANGE BETWEEN INTERVAL '7 days' PRECEDING AND CURRENT ROW) AS transactions_in_last_7d,
        AVG(amt) OVER (w RANGE BETWEEN INTERVAL '7 days' PRECEDING AND CURRENT ROW) AS avg_amt_in_last_7d,
        
        amt / NULLIF(AVG(amt) OVER (w RANGE BETWEEN INTERVAL '7 days' PRECEDING AND CURRENT ROW), 0) AS pct_of_avg_amt_in_last_7d,
        amt / NULLIF(AVG(amt) OVER (w RANGE BETWEEN INTERVAL '24 hours' PRECEDING AND CURRENT ROW), 0) AS pct_of_avg_amt_in_last_24h
        
    FROM
        transactions
    WINDOW w AS (
        PARTITION BY cc_num
        ORDER BY (trans_date || ' ' || trans_time)::timestamp
    )
    ORDER BY
        cc_num, trans_ts
    LIMIT 100;
""")

df_frequency = pd.read_sql_query(sql=query_f, con=engine)
df_frequency.head(20)

,cc_num,trans_ts,amt,time_since_last_transaction_sec,transactions_in_last_24h,transactions_in_last_6h,avg_amt_in_last_24h,transactions_in_last_7d,avg_amt_in_last_7d,pct_of_avg_amt_in_last_7d,pct_of_avg_amt_in_last_24h
0,60400268763,2024-01-01 02:34:47,59.39,NaN,1,1,59.390000,1,59.390000,1.000000,1.000000
1,60400268763,2024-01-01 09:56:16,8.33,26489.0,2,1,33.860000,2,33.860000,0.246013,0.246013
2,60400268763,2024-01-04 05:09:17,7.51,241981.0,1,1,7.510000,3,25.076667,0.299482,1.000000
3,60400268763,2024-01-04 08:59:03,12.42,13786.0,2,2,9.965000,4,21.912500,0.566800,1.246362
4,60400268763,2024-01-06 09:04:15,64.03,173112.0,1,1,64.030000,5,30.336000,2.110694,1.000000
5,60400268763,2024-01-07 01:18:13,2.98,58438.0,2,1,33.505000,6,25.776667,0.115608,0.088942
6,60400268763,2024-01-07 10:18:50,106.24,32437.0,2,1,54.610000,7,37.271429,2.850441,1.945431
7,60400268763,2024-01-09 02:31:45,43.13,144775.0,1,1,43.130000,6,39.385000,1.095087,1.000000
8,60400268763,2024-01-09 08:25:07,67.34,21202.0,2,2,55.235000,7,43.378571,1.552379,1.219155
9,60400268763,2024-01-11 04:00:29,5.63,156922.0,1,1,5.630000,8,38.660000,0.145629,1.000000


## 2. Customers' Atypical Behavior
顧客の平常時からの逸脱を数値化
- 過去全体の平均取引額
- ↑と現在の取引額の比
- カテゴリごとの購入の頻度: いつも使わないカテゴリでの買い物は怪しい？

In [24]:
query_ab = text("""
    SELECT
        cc_num,
        (trans_date || ' ' || trans_time)::timestamp AS trans_ts,
        amt,
        category,
        
        -- Overall average amount
        AVG(amt) OVER(w ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS avg_amt_historical,
        
        -- Ratio to overall average
        amt / NULLIF(AVG(amt) OVER(w ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING), 0) AS amt_ratio_to_avg,
        
        -- How many transactions in certain category so far
        COUNT(*) OVER (PARTITION BY cc_num, category ORDER BY (trans_date || ' ' || trans_time)::timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS category_tx_count
        
    FROM
        transactions
    WINDOW w AS (
        PARTITION BY cc_num
        ORDER BY (trans_date || ' ' || trans_time)::timestamp
    )
    ORDER BY
        cc_num, trans_ts
    LIMIT 100;
""")

df_ab = pd.read_sql_query(sql=query_ab, con=engine)
df_ab.head(20)

,cc_num,trans_ts,amt,category,avg_amt_historical,amt_ratio_to_avg,category_tx_count
0,60400268763,2024-01-01 02:34:47,59.39,gas_transport,NaN,NaN,0
1,60400268763,2024-01-01 09:56:16,8.33,gas_transport,59.390000,0.140259,1
2,60400268763,2024-01-04 05:09:17,7.51,gas_transport,33.860000,0.221796,2
3,60400268763,2024-01-04 08:59:03,12.42,gas_transport,25.076667,0.495281,3
4,60400268763,2024-01-06 09:04:15,64.03,gas_transport,21.912500,2.922076,4
5,60400268763,2024-01-07 01:18:13,2.98,gas_transport,30.336000,0.098233,5
6,60400268763,2024-01-07 10:18:50,106.24,gas_transport,25.776667,4.121557,6
7,60400268763,2024-01-09 02:31:45,43.13,gas_transport,37.271429,1.157187,7
8,60400268763,2024-01-09 08:25:07,67.34,gas_transport,38.003750,1.771930,8
9,60400268763,2024-01-11 04:00:29,5.63,gas_transport,41.263333,0.136441,9


## 3. Short Term Txs in Same Merchant

In [25]:
query_sm = text("""
    SELECT
        cc_num,
        (trans_date || ' ' || trans_time)::timestamp AS trans_ts,
        merchant,
        
        -- How many transactions in certain merchant in 1 hour
        COUNT(*) OVER (
            PARTITION BY cc_num, merchant
            ORDER BY (trans_date || ' ' || trans_time)::timestamp
            RANGE BETWEEN INTERVAL '1 hour' PRECEDING AND CURRENT ROW
        ) AS tx_with_same_merchant_in_last_1h
        
    FROM
        transactions
    ORDER BY
        cc_num, trans_ts
    LIMIT 100;
""")

df_sm = pd.read_sql_query(sql=query_sm, con=engine)
df_sm.head(20)

,cc_num,trans_ts,merchant,tx_with_same_merchant_in_last_1h
0,60400268763,2024-01-01 02:34:47,"fraud_Greenholt, Jacobi and Gleason",1
1,60400268763,2024-01-01 09:56:16,fraud_Torp-Labadie,1
2,60400268763,2024-01-04 05:09:17,fraud_Bernhard Inc,1
3,60400268763,2024-01-04 08:59:03,"fraud_Olson, Becker and Koch",1
4,60400268763,2024-01-06 09:04:15,fraud_Marks Inc,1
5,60400268763,2024-01-07 01:18:13,"fraud_Connelly, Reichert and Fritsch",1
6,60400268763,2024-01-07 10:18:50,"fraud_Lind, Huel and McClure",1
7,60400268763,2024-01-09 02:31:45,fraud_Kling Inc,1
8,60400268763,2024-01-09 08:25:07,fraud_Parisian and Sons,1
9,60400268763,2024-01-11 04:00:29,"fraud_Stroman, Hudson and Erdman",1
